In [92]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from beakerx import *
plt.style.use('ggplot')
import seaborn as sns
sns.set()
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from scipy.io import loadmat
from matplotlib.mlab import griddata
tableau20 = np.array([(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)])/255.
styles = ["white","dark",'whitegrid',"darkgrid"]
contexts = ['paper','talk','poster','notebook']
from ipywidgets import interact, FloatSlider,IntSlider, RadioButtons,Dropdown,Tab,Text,SelectMultiple,FloatRangeSlider
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from mpl_toolkits.mplot3d.axes3d import *
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler,Imputer
from sklearn.feature_selection import VarianceThreshold
import argparse
import numba as nb

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['Text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
/home/xupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
feature_counts = 10
point_counts = 1000
test_counts = 1000

In [4]:
rawdata = loadmat('/home/xupeng/projects/signal/data/citydata.mat')
RSS = rawdata['RSS']
TOA = rawdata['TOA']
DOA = rawdata['DOA']
TXloc = rawdata['TX']
RXloc = rawdata['RX']
xyzorigin_RX = np.array([np.min(RXloc[:,0]),np.min(RXloc[:,1]),0])
xyzorigin_TX = np.array([np.min(TXloc[:,0]),np.min(TXloc[:,1]),0])
RXloc_shifted = RXloc - xyzorigin_RX
TXloc_shifted = TXloc - xyzorigin_TX
distance=np.zeros((1500,1500))
for i in tqdm(range(1500)):
    for j in range(1500):
        distance[i,j]=np.linalg.norm(RXloc_shifted[i]-RXloc_shifted[j])
#distance=np.loadtxt('/home/xupeng/projects/signal/citydata/distance.txt',dtype=np.float32)
c=np.append(RSS,TOA,axis=1)
combine_data=np.append(c,DOA,axis=1)

In [90]:
def relate(index_1, index_2, features):
    feature_rss_toa=features[features>=0]
    feature_doa=features[features<0]
    feature_doa=-feature_doa-1
    feature_doax=feature_doa*3+200
    feature_doay=feature_doax+1
    feature_doaz=feature_doax+2
    feature_doaxy=np.append(feature_doax,feature_doay)
    feature_doa=np.append(feature_doaxy,feature_doaz)
    features=np.append(feature_rss_toa,feature_doa)
    xdata=combine_data[index_1]
    xdata=xdata[features]
    ydata=combine_data[index_2]
    ydata=ydata[features]
    an=~np.isnan(xdata)&~np.isnan(ydata)
    xdata=xdata[an]
    nr=1-len(xdata)/len(ydata)
    if len(xdata) == 0:
        return np.nan,1
    ydata=ydata[an]
    an=an[an]
    r=scipy.stats.pearsonr(xdata,ydata)[0]
    return r,nr

In [85]:
def corelate(features):
    batchsize=100;
    r=np.zeros(testnum)
    nr=np.zeros(testnum)
    xn=np.random.randint(1,1500,size=(testnum,batchsize))
    yn=np.random.randint(1,1500,size=(testnum,batchsize))
    r=np.zeros((testnum,batchsize))
    u=np.zeros((testnum,batchsize))
    d=np.zeros((testnum,batchsize))
    re=np.zeros(testnum)
    nr=np.zeros(testnum)
    for j in range(testnum):
        nan_num=0;
        for i in range(batchsize):
            d[j][i]=distance[xn[j][i],yn[j][i]]
            r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
            while np.isnan(r[j][i]):
                xn[j][i]=np.random.randint(1,1500)
                yn[j][i]=np.random.randint(1,1500)
                d[j][i]=distance[xn[j][i],yn[j][i]]
                r[j][i],u[j][i]=relate(xn[j][i],yn[j][i],features)
                nan_num=nan_num+1
        re[j]=scipy.stats.pearsonr(d[j],r[j])[0]
        nr[j]=(np.sum(u[j])+nan_num)/(nan_num+batchsize)
    return -np.mean(re), np.mean(nr), xn.flatten(), yn.flatten(), r.flatten(), u.flatten()

In [97]:
test_counts=1000
point_counts=1000
feature_counts=10


In [7]:
batchsize=100;
testnum=int(point_counts/batchsize)
x_total=np.zeros((test_counts,testnum*batchsize))
y_total=np.zeros((test_counts,testnum*batchsize))
r_total=np.zeros((test_counts,testnum*batchsize))
u_total=np.zeros((test_counts,testnum*batchsize))
relation=np.zeros(test_counts)

label=np.random.randint(-100,200,size=(test_counts,feature_counts))

nan_rate=np.zeros(test_counts)


In [8]:
for i in tqdm(range(test_counts)):
    relation[i],nan_rate[i],x_total[i],y_total[i],r_total[i],u_total[i]=corelate(label[i])


  0%|          | 0/1000 [00:00<?, ?it/s]/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3006: RuntimeWarning: Mean of empty slice.
  my = y.mean()
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3020: RuntimeWarning: invalid value encountered in double_scalars
  prob = _betai(0.5*df, 0.5, df/(df+t_squared))
100%|██████████| 1000/1000 [00:41<00:00, 24.22it/s]


In [9]:
index=np.argsort(-relation)
relation=relation[index]
label=label[index]
nan_rate=nan_rate[index]
x_total=x_total[index]
y_total=y_total[index]
u_total=u_total[index]
r_total=r_total[index]

In [84]:
dir_name=('__DOA__'+str(feature_counts)+'feature_counts'+str(test_counts)+'test_counts'+str(point_counts)+'point_counts')
path_model = os.path.join('/home/xupeng/projects/signal/citydata/', dir_name)
if not os.path.exists(path_model):
    os.makedirs(path_model)

100%|██████████| 1500/1500 [00:15<00:00, 99.13it/s]


In [91]:
np.savetxt(path_model+'/pcc.txt',relation)
np.savetxt(path_model+'/label.txt',label,fmt='%d')
np.savetxt(path_model+'/nan_rate.txt',nan_rate,fmt='%4.3f')
np.savetxt(path_model+'/first_index.txt',x_total,fmt='%d')
np.savetxt(path_model+'/second_index.txt',y_total,fmt='%d')
np.savetxt(path_model+'/nanrate_specific.txt',u_total,fmt='%3.2f')
np.savetxt(path_model+'/relation_specific.txt',r_total)

/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3020: RuntimeWarning: invalid value encountered in double_scalars
  prob = _betai(0.5*df, 0.5, df/(df+t_squared))
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3005: RuntimeWarning: Mean of empty slice.
  mx = x.mean()
/home/xupeng/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/xupeng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3006: RuntimeWarning: Mean of empty slice.
  my = y.mean()


(0.8337431414470013, 0.3101109743382774)

In [99]:
weight=np.ones(500)
weight=weight/np.sum(weight)
label=np.random.choice(500,size=(test_counts,feature_counts),p=weight)


In [101]:
label.shape

(1000, 10)

In [102]:
label=np.random.choice(500,replace=False,size=(feature_counts),p=weight)

ValueError: Cannot take a larger sample than population when 'replace=False'